# Extract and display an adjacency matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 01.2025

## Summary
This analysis extracts and visualizes the connectivity between all pairs of pre- and post-synaptic neurons (adjacency matrix), optionally including the number of synapses per connection (synaptome matrix).
For details, see the [README](README.md).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import adjacency
from ipywidgets import widgets, interact

## Circuit selection

A SONATA circuit is assumed to be located under `./analysis_circuit/circuit_config.json`. For using an existing circuit from some other location, modify the `circuit_config = ...` path below accordingly.

In [2]:
# Path to existing circuit config
circuit_config = "analysis_circuit/circuit_config.json"

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [3]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

Dropdown(description='Edge population:', layout=Layout(width='max-content'), options=('nodeA__nodeA__chemical'…

Dropdown(description='Pre-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', 'L…

Dropdown(description='Post-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', '…

## Adjacency matrix extraction

Extract the adjacency/synaptome matrix using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [4]:
adj_syn_dict = adjacency.compute(c, sel_src=pre_nset_wdgt.value, sel_dest=post_nset_wdgt.value, edges_popul_name=e_popul_wdgt.value)

INFO: Creating 10x10 adjacency matrix (sel_src=None, sel_dest=None)


## Interactive visualization

Interactive visualization of adjacency or synaptome matrix. Marker scaling and alpha (transparency) can be adjusted for better display of small/large matrices.

In [5]:
# Interactive plot function
def plot_fct(res_sel, mscale, alpha):
    res_dict = adj_syn_dict[res_sel]
    cmap = "hot_r"

    mat = res_dict["data"].tocoo()  # Convert to COO, for easy access to row/col and data!!
    col_idx = mat.data
    vmin = 0
    vmax = max([1, *col_idx])
    ms = mscale * 5000 / max(mat.shape),  # Adjust marker size for proper display

    plt.figure()
    plt.scatter(mat.col, mat.row, marker=",", s=ms, edgecolors="none", alpha=alpha, c=col_idx, cmap=cmap, vmin=vmin, vmax=vmax, label="Conn")

    plt.xlabel("Post-neurons" + ("" if post_nset_wdgt.value is None else f" [{post_nset_wdgt.value}]"))
    plt.ylabel("Pre-neurons" + ("" if pre_nset_wdgt.value is None else f" [{pre_nset_wdgt.value}]"))
    plt.title(res_sel_wdgt.label, fontweight="bold")

    plt.axis("image")
    plt.xlim((-0.5, res_dict["data"].shape[1] - 0.5))
    plt.ylim((-0.5, res_dict["data"].shape[0] - 0.5))
    plt.gca().invert_yaxis()
    if res_dict["data"].dtype != bool:
        cb = plt.colorbar(label="#Synapses")
        cb_ticks = np.unique(np.round(cb.get_ticks()))
        cb.set_ticks(cb_ticks[(cb_ticks >= vmin) & (cb_ticks <= vmax)])
    plt.tight_layout()
    plt.show()

In [6]:
res_sel_wdgt = widgets.Dropdown(options=[("Adjacency matrix", "adj"), ("Synaptome matrix", "adj_cnt")], description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
mscale_wdgt = widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=1, step=0.01, description="Marker scale:", style={"description_width": "auto"})
alpha_wdgt = widgets.FloatSlider(value=1.0, min=0.01, max=1.0, step=0.01, description="Marker alpha:", style={"description_width": "auto"})
iplot = interact(plot_fct, res_sel=res_sel_wdgt, mscale=mscale_wdgt, alpha=alpha_wdgt)

interactive(children=(Dropdown(description='Display:', layout=Layout(width='max-content'), options=(('Adjacenc…